<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/paths_InPython.ipynb" target="_blank">
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Paths in Python 📒
***Working with Paths in Python: os module vs Shell Commands***

by Silvia Mazzoni, DesignSafe, 2025

When writing scripts that manipulate files or navigate directories, it’s critical to use tools that are **portable, robust, and easy to maintain across systems**.

On DesignSafe (and generally on HPC systems), your scripts might run on JupyterHub, a virtual machine, or a batch environment—so writing **path operations in pure Python** ensures they behave consistently everywhere.


In [1]:
import os

## **os** Python Module

The *os.path* module is part of Python’s built-in *os* library and helps you **work with file system paths in a clean, portable way**. Instead of relying on system-specific shell commands like *cd*, *pwd*, or *ls*, *os.path* and related *os* functions let you write code that works consistently across **Linux, macOS, and Windows**.

Unlike shell commands, which only **display output to the screen**, Python’s *os* and *os.path* functions **return actual values** (like strings, lists, or booleans) that you can use directly in your code. For example, *os.listdir()* gives you a list of filenames you can iterate over—no parsing, no copy-pasting from the terminal.

When comparing options:

* *os* and *os.path* are **Python-native** — safe, readable, and portable.
* Shell commands like *ls* or *cd* are **fragile** and don’t return data you can use in Python.
* *subprocess* is powerful, but better suited for running full programs or system-level tools, not basic path operations.

If you're building a workflow that needs to run reliably in different environments—like JupyterHub, VMs, or HPC systems—use *os.path*.

### Why use *os.path*?

* **Portable:** handles differences between Linux, macOS, and Windows automatically (slashes, etc.).
* **Works with Python variables:** no need to parse command output.
* **Safe and predictable:** your scripts don’t depend on external shell programs.

By contrast:

* Shell commands via **os.system()** are less predictable and can’t return values to Python variables.
* The **subprocess** module is very powerful for calling external programs (like running OpenSees or SLURM scripts), but it’s overkill for simple file operations.


## Examples with *os* and *os.path*

| Task                               | Recommended Python Code                   | Shell Equivalent            |
| ---------------------------------- | ----------------------------------------- | --------------------------- |
| Get current directory              | `os.getcwd()`                             | `pwd`                       |
| List files in a directory          | `os.listdir(path)`                        | `ls path`                   |
| Change directory                   | `os.chdir(path)`                          | `cd path`                   |
| Expand `~` to home directory       | `os.path.expanduser('~')`                 | `echo ~`                    |
| Build safe path                    | `os.path.join(dir, file)`                 | `cd dir/subdir` *(fragile)* |
| Get absolute path from relative    | `os.path.abspath('myfile.txt')`           | `realpath myfile.txt`       |
| Check if a path exists             | `os.path.exists(path)`                    | `test -e path` or `ls`      |
| Check if path is file or directory | `os.path.isfile(path)`, `os.path.isdir()` | `file path`, `ls -ld path`  |
| Create a directory                 | `os.mkdir(path)`                          | `mkdir path`                |
| Create all parent directories      | `os.makedirs(path, exist_ok=True)`        | `mkdir -p path`             |
| Rename/move a file                 | `os.rename('old', 'new')`                 | `mv old new`                |
| Remove a file                      | `os.remove('file.txt')`                   | `rm file.txt`               |
| Remove an empty directory          | `os.rmdir('folder')`                      | `rmdir folder`              |
| Remove a directory (recursively)   | `shutil.rmtree('folder')`                 | `rm -r folder`              |
| Copy a file                        | `shutil.copy('src', 'dst')`               | `cp src dst`                |
| Copy a directory                   | `shutil.copytree('src', 'dst')`           | `cp -r src dst`             |

✅ **Tip:** Use the `os` and `os.path` modules for general-purpose, portable path and file manipulation. Use `shutil` when you need higher-level operations like copying or deleting directories -- see below for more on shutil.

---
## Check if a path exists
***Checking if a path exists with os.path.exists***

Before working with any file or directory — such as listing its contents, reading a file, or writing data — it’s good practice to first **check whether the path actually exists on the filesystem**.

Python’s `os.path` module provides a simple way to do this:

### Why this matters

* Running commands on paths that **don’t exist** often causes errors like `FileNotFoundError`.
* This is especially important on DesignSafe, where **storage may be mounted differently** on JupyterHub, the OpenSees VM, or Stampede3. A path valid on one system might not exist on another.
* Checking existence first makes your scripts more robust and user-friendly — you can print helpful messages or even create missing directories.

Adding `os.path.exists` checks into your workflow helps you avoid many common bugs and ensures your code handles different environments gracefully.

In [2]:
# initialize:
os.chdir(os.path.expanduser('~')) # start at the home directory, we will discuss these commands

In [3]:
# Check if path exists:
thisPath = 'MyData'

if os.path.exists(thisPath):
    print(f"The path exists: {thisPath}")
else:
    print(f"Path does not exist: {thisPath}")

The path exists: MyData


---
## Absolute vs Relative Path
Yes, that’s a good general rule—especially for Unix-like systems (like Linux and macOS), including the HPC systems on DesignSafe. Here's how you can phrase it more precisely:

> An **absolute path** starts with a `/` and specifies the full location of a file or directory from the root of the file system.
>
> A **relative path** does **not** start with a `/` and is interpreted in relation to the current working directory (e.g., `.`).

### Example:

```bash
# Absolute path
/data/user/simulation/input.tcl

# Relative path (from current directory)
../input.tcl
./scripts/run.sh
```

> ⚠️ On Windows, absolute paths start with a drive letter like `C:\`, so this rule is Unix-specific—but totally valid on DesignSafe.


---
## expanduser and abspath
***Getting the full absolute path***

You can define the full path with or without **~** (a special UNIX shorthand for your home directory).

Let’s make sure you can build **robust absolute paths**, no matter how the path is specified.

### Why this matters:
- Paths without **~** are interpreted relative to your current working directory.  
- Different environments on DesignSafe mount your storage under different root directories. If you rely on relative paths without care, your code might break or point to unexpected locations.


### Using *expanduser* and *abspath* together

- **os.path.expanduser(path)** replaces **~** with your actual home directory.  
- **os.path.abspath(path)** resolves any relative path (like *MyData*) to a full path from **/**, based on where you’re currently working.

But each does only half the job:

### expanduser

In [4]:
os.path.expanduser('~/MyData') # replaces ~ with your actual home directory

'/home/jupyter/MyData'

In [5]:
os.path.exists(os.path.expanduser('~/MyData'))

True

In [6]:
os.path.expanduser('/MyData')

'/MyData'

In [7]:
os.path.exists(os.path.expanduser('/MyData'))

False

In [8]:
os.path.expanduser('MyData') # (unchanged, still relative)

'MyData'

In [9]:
os.path.exists(os.path.expanduser('MyData'))

True

### abspath

In [10]:
os.path.abspath('MyData') # (convert relative to absolute, based on cwd)

'/home/jupyter/MyData'

In [11]:
os.path.exists(os.path.abspath('MyData'))

True

In [12]:
os.path.abspath('~/MyData') ## gives unrealistic results

'/home/jupyter/~/MyData'

In [13]:
os.path.exists(os.path.abspath('~/MyData'))

False

### Combine abspath  expanduser
The safest pattern is to **combine them**. This way, your path works whether it starts with **~** or is just relative:

In [14]:
os.path.abspath(os.path.expanduser('~/MyData'))

'/home/jupyter/MyData'

In [15]:
os.path.exists(os.path.abspath(os.path.expanduser('~/MyData')))

True

In [16]:
os.path.abspath(os.path.expanduser('MyData')) # this is a relative path, so it works only when MyData exists

'/home/jupyter/MyData'

In [17]:
os.path.exists('MyData')

True

In [18]:
os.path.exists(os.path.abspath(os.path.expanduser('MyData')))

True

#### Rule of thumb

* Always use `expanduser` to safely expand `~`.
* Always follow it with `abspath` to ensure you get a full path from `/`.

This tiny two-step makes your code **portable and predictable**—especially across JupyterHub, the OpenSees VM, Stampede3, and Tapis, where your storage may appear under different absolute root directories.

Understanding how to combine `expanduser` and `abspath` ensures your scripts will find your data reliably, saving you from countless `FileNotFoundError` surprises—no matter which system you’re running on.

### Summary of absolute and relative paths
- Always use **absolute paths** for reliability.  
- Use `os.path.expanduser("~")` to safely handle home directories.  
- Prefer `os.path` functions over shell commands for **portability across DesignSafe systems**.  
- Remember: the same storage might appear differently in JupyterHub, the OpenSees VM, Stampede3, and Tapis — understanding paths keeps your workflows smooth and error-free.


---
## Root & Home

When working in Jupyter environments, the **full absolute path is often hidden behind a simplified interface**. You typically start navigating at directories like *MyData*, *Work*, or *MyProjects*, without seeing the full path that comes before them. However, these base paths do exist—and they vary depending on the system you’re using.

On DesignSafe, this is especially important because **the same storage systems appear differently across platforms**. For example:

* JupyterHub mounts **MyData** under **/home/jupyter/MyData/**
* Stampede3 doesn’t mount **MyData** at all, but uses your **\$HOME** directory instead, which is part of a different storage system.
* Tapis uses URI-style paths like **tapis\://designsafe.storage.default/username/** (more on this later...)

Understanding these differences is key to writing scripts and workflows that are portable and error-free across all of DesignSafe’s environments.

### The root directory (`/`) vs. your home directory (`~`)

When talking about file systems on UNIX-like systems (including Linux, macOS, and the systems powering JupyterHub and Stampede3), it’s important to distinguish between:

* **The root directory `/`**
    This is the **top of the entire file system hierarchy**. Everything on the machine—system files, programs, shared data, and user directories—lives somewhere under **/**.  

    Examples of absolute paths from root include::
    ```
    /
    /home
    /scratch
    /usr/bin
    ```

* **Your home directory `~`**
    This is a **shortcut to your personal user space**, sometimes called your “home directory.”
    It’s where your files, configurations, and most project work reside. Typical absolute equivalents look like:
    ```
    /home/silvia
    or
    /Users/silvia
    ````

In other words, `/` is the root for **the whole machine**, while `~` is the root for **your own user environment**. That’s why `~` is sometimes described as “the root of your personal space.”

### Why does this matter?

* When you use `/` to start a path, you’re telling the system to look from the very top. For example:

  ```
  /usr/bin/python
  /scratch/myproject/output.txt
  ```

  These are **system-level paths**—they exist independently of who’s logged in.

* When you use `~`, you’re telling the system to look in **your own user directory**. For example:

  ```
  ~/my-scripts/run-opensees.sh
  ~/results/output.dat
  ```

  These are **specific to your user**, making them portable and safer (you won’t accidentally mess with system files).

### Example with `/` in Python

In [19]:
thisPath = '/'
expanded = os.path.abspath(os.path.expanduser(thisPath))

print(f"Path: {thisPath}")
print(f"Expanded: {expanded}")

Path: /
Expanded: /


In [20]:
AllContents = os.listdir(expanded)
print(f"\n Get contents using: os.listdir('{thisPath}'): {AllContents}")


 Get contents using: os.listdir('/'): ['media', 'home', 'sbin', 'proc', 'boot', 'opt', 'tmp', 'usr', 'mnt', 'dev', 'var', 'sys', 'srv', 'bin', 'etc', 'run', 'lib', 'root', 'lib64', 'sbin.usr-is-merged', 'bin.usr-is-merged', 'lib.usr-is-merged']


### Example with `~` in Python

In [21]:
Path = '~'
expanded = os.path.expanduser(Path)

print(f"Path: {Path}")
print(f"Expanded: {expanded}")

Path: ~
Expanded: /home/jupyter


In [22]:
AllContents = os.listdir(expanded)
print(f"\n Get contents using: os.listdir('{Path}'): {AllContents}")


 Get contents using: os.listdir('~'): ['.profile', '.bashrc', '.bash_logout', '.tapis_tokens.json', '.python_history', '.local', '.bash_history', '.ipython', '.cache', 'results', 'Work', '.config', '.tapis-token', 'MyProjects', 'CommunityData', 'NEES', 'OutFiles_4dfa35e1-15cd-48fd-a090-f348544dee1f-007', 'NHERI-Published', 'MyData', '.jupyter', '.wget-hsts', '.julia', '.matlab']


This shows that `"~"` is **not the same as `/`**—it expands to your personal home directory.


| Symbol | Means                             |
| ------ | --------------------------------- |
| `/`    | **root of the entire filesystem** |
| `~`    | **your user’s home directory**    |



### In practice

Understanding this difference is crucial when writing absolute paths.

* If you use `/`, you start from the very top of the machine’s filesystem. This is required when referencing global locations or shared directories.

* If you use `~`, you start from your personal space. This is safer and more portable—especially across different systems on DesignSafe—since it doesn’t depend on system-level structures that might change.

This distinction also helps you avoid mistakes like trying to list or modify system directories (`/etc`, `/usr`) when you really meant to work inside your own files.

---
## Getting your current path

Because different environments mount storage systems differently, querying the current path (**"."**) may show different results.

To get the **absolute path of your current directory**, you can use:

In [23]:
os.getcwd()

'/home/jupyter'

In [24]:
os.path.abspath('.')           # same result, more general

'/home/jupyter'

Both will typically give the same absolute path. In this notebook, we’ll change directories into each storage system (and a subfolder) and query **"."** to illustrate how paths differ across environments.

---
## Creating Directories

When working with file outputs or organizing your data, it’s common to need a specific directory. Rather than assuming it exists, your script can **check and create it if necessary**.

Use `os.path.exists()` to check whether a directory is already present, and `os.makedirs()` to create it if it’s missing.

In [25]:
thisPath = 'results/output_data'

if not os.path.exists(thisPath):
    print(f"Creating missing directory: {thisPath}")
    os.makedirs(thisPath)
else:
    print(f"Directory already exists: {thisPath}")

Directory already exists: results/output_data


### Why use `os.makedirs()`?

* It **creates intermediate directories** as needed (e.g., `a/b/c` even if `a` and `b` don’t exist yet).
* It’s safe to use in automated workflows where directory structure might vary.
* Use it **before writing files** to ensure your destination exists.

> ✅ Tip: If you use Python 3.2 or newer, you can also add `exist_ok=True` to skip the existence check:

In [26]:
thisPath = 'temporalxxx'
os.makedirs(thisPath, exist_ok=True); 

This makes your script **more robust and portable**, especially in shared or multi-user environments like DesignSafe.

---
## Deleting Files and Directories
***deleting files and directories safely using Python***

Sometimes you’ll want to **clean up old files**, remove temporary data, or ensure a directory is empty before writing new output. Python provides built-in tools for this, mostly through the `os` and `shutil` modules.

### Deleting a file

In [27]:
file_path = 'output/log.txt'

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Deleted file: {file_path}")
else:
    print(f"File not found: {file_path}")

File not found: output/log.txt


### Deleting an Empty Directory

Use `os.rmdir()` to delete a directory **only if it's empty**:

In [28]:
thisPath = 'temporalxxx'
if os.path.exists(thisPath):
    try:
        os.rmdir(thisPath)
        print(f"Deleted empty directory: {thisPath}")
    except OSError:
        print(f"Directory not empty: {thisPath}")

Deleted empty directory: temporalxxx


### Deleting a Directory and All Its Contents

To delete a directory **and everything inside it**, use `shutil.rmtree()`:

In [29]:
import shutil

dir_path = 'results/temp'

if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
    print(f"Deleted directory and all contents: {dir_path}")
else:
    print(f"Directory not found: {dir_path}")

Directory not found: results/temp


> ⚠️ **Warning:** `shutil.rmtree()` is **not reversible**. Always double-check the path before using it to avoid deleting critical data.

---

This gives you full control over cleaning up files and directories in your workflow — whether you're preparing for a new simulation run or automating post-processing steps.


## Why Use `shutil`?

While `os` and `os.path` handle **basic file and path operations**, the `shutil` module provides **higher-level utilities** for working with files and directories — especially when you need to:

* **Copy** files or folders
* **Delete entire directories** (recursively)
* **Move or rename** large data structures

It's a powerful tool for scripting workflows that manipulate lots of data, such as preparing input folders, archiving results, or cleaning up temporary files.

---

### Common `shutil` Functions

| Task                             | Python Code                         | Description                                     |
| -------------------------------- | ----------------------------------- | ----------------------------------------------- |
| Copy a file                      | `shutil.copy('src.txt', 'dst.txt')` | Copies the contents and metadata of a file      |
| Copy a directory (recursively)   | `shutil.copytree('src/', 'dst/')`   | Copies a full folder structure and contents     |
| Move or rename a file/folder     | `shutil.move('old', 'new')`         | Moves or renames files or folders               |
| Delete a directory (recursively) | `shutil.rmtree('folder')`           | Removes a directory and all its contents        |
| Disk usage of a path             | `shutil.disk_usage('/')`            | Shows total, used, and free space at a location |
| Create archive (zip/tar)         | `shutil.make_archive(...)`          | Builds a `.zip`, `.tar`, or `.gztar` archive    |

---

✅ **Note:** `shutil` is part of the Python Standard Library — no installation needed!


---
## Changing Directories
***Changing your working directory with os.chdir***

Once you’ve found the path you want to work in, you can **change your current working directory** so that all relative operations happen there.  
This is done using Python’s:

In [30]:
print('current location',os.getcwd())

thisPath = os.path.expanduser('~/MyData')
os.chdir(thisPath)

print('new location',os.getcwd())

current location /home/jupyter
new location /home/jupyter/MyData


From this point on, any command using a relative path (like `"./somefile.txt"`) will look inside this directory.

### Always check that the directory exists first

Before changing directories, it’s good practice to make sure the path actually exists.
Otherwise, *os.chdir* will throw a *FileNotFoundError*.

In [31]:
print('current location',os.getcwd())

thisPath = os.path.abspath(os.path.expanduser('~/YourData'))

if os.path.exists(thisPath):
    os.chdir(thisPath)
    print(f"Changed working directory to: {thisPath}")
    print(f"Current working directory is now: {os.getcwd()}")
else:
    print(f"Path does not exist: {thisPath}")
    
print('new location',os.getcwd())

current location /home/jupyter/MyData
Path does not exist: /home/jupyter/YourData
new location /home/jupyter/MyData


### Why change directories?

* Lets you keep your relative paths simple.
* Avoids having to write out long absolute paths repeatedly.
* Makes it easier to run scripts or load data, knowing they’ll look in this directory.

### A few tips

* Always print your new location with **os.getcwd()** after changing directories — this avoids confusion about where your notebook or script is “pointing.”
* Remember this change is **per-process**: it only affects the current Python session.

Using *os.chdir* along with *os.path.exists*, *os.getcwd*, and your robust path-building ensures your notebooks are **safe, portable, and clear across all DesignSafe platforms.**


---
## Directory Contents
***Listing files and directories with `os.listdir`***

Once you’ve confirmed that a path exists, you can explore its contents using Python.  
The simplest way to do this is with **os.listdir**, which returns a list of all files and folders inside the specified directory.

### Example usage

In [32]:
thisPath =os.path.abspath(os.path.expanduser('~/MyData'))

print('thisPath:',thisPath)

thisPath: /home/jupyter/MyData


### All Contents

In [33]:
AllContents = os.listdir(thisPath)
print(f"\n Get contents using: os.listdir('{thisPath}'): {AllContents}")


 Get contents using: os.listdir('/home/jupyter/MyData'): ['.Trash', '.ipynb_checkpoints', '.jupyter', '210219_10StorySapModel', 'BBPsimsData_180521', 'Baker_CS_Selection', 'Centrifuge_notebook', 'FEMAP695records', 'Frankel_etal_Production', 'NGAeastGMM', 'NGArecords', 'NGAsubGMMtool', 'PrivateProjectData', 'RCTC', 'SCEC_BBP_GMportal', 'SCEC_BBP_Study_17_3_180625', 'SpectraData', 'Temp', 'Utilities', 'archive', 'gmfiles', 'FEMAP695records.zip', 'Frankel_etal_Production.zip', 'Untitled.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled3.ipynb', 'Testings', 'test.ipynb', 'eSEESminiPy', 'JupyterApps', 'template-opensees-freefield.ipynb', 'opensees2.ipynb', 'UseCases', 'BeamColumModelingInOpenSees_2_Sections.ipynb', 'tapis-jobs-archive', 'SFSI_ATC140ssi_superceded', 'OpenSees', 'GroundMotions', 'AgaveAPI_Interface_V1.ipynb', 'NHR3', '.opensees-interactive.bak', 'jupyter-templates', '.opensees-interactive.bak2', '.opensees-interactive', 'OpenSeesMPJupyter-TAPISV3silvia.ipynb', 'TapisV3

This prints a simple list of names (both files and directories) found under `thisPath`.

### Splitting into files and directories

To organize this output more clearly, you can separate **files** and **directories** using list comprehensions and **os.path.isdir**:

In [34]:
directories = [name for name in AllContents if os.path.isdir(os.path.join(thisPath, name))]

print(f" -- directories:",directories[:20]); # show only the first few

 -- directories: ['.Trash', '.ipynb_checkpoints', '.jupyter', '210219_10StorySapModel', 'BBPsimsData_180521', 'Baker_CS_Selection', 'Centrifuge_notebook', 'FEMAP695records', 'Frankel_etal_Production', 'NGAeastGMM', 'NGArecords', 'NGAsubGMMtool', 'PrivateProjectData', 'RCTC', 'SCEC_BBP_GMportal', 'SCEC_BBP_Study_17_3_180625', 'SpectraData', 'Temp', 'Utilities', 'archive']


In [35]:
files = [name for name in AllContents if not os.path.isdir(os.path.join(thisPath, name))] # select files as not directories

print(f" -- files:",files[:20]); # show only the first few

 -- files: ['FEMAP695records.zip', 'Frankel_etal_Production.zip', 'Untitled.ipynb', 'Untitled1.ipynb', 'Untitled2.ipynb', 'Untitled3.ipynb', 'test.ipynb', 'template-opensees-freefield.ipynb', 'opensees2.ipynb', 'BeamColumModelingInOpenSees_2_Sections.ipynb', 'AgaveAPI_Interface_V1.ipynb', 'OpenSeesMPJupyter-TAPISV3silvia.ipynb', 'study_22_12_lf_indexed.sqlite', 'tmp_tapis_submitJob_WebPortalApp.ipynb', 'tmp_tapis_submitJob_WebPortalApp_Def.ipynb', 'tmp_tapis_submitJob_WebPortalApp_Auto.ipynb']


### Why this is helpful

* You get a quick overview of what’s inside your data folder.
* Separating files vs. directories is especially useful if your scripts process only data files, or need to recursively explore subfolders.
* **The list of files is stored in an array so you can use it in your workflow**

---
##  Building Paths with `os.path.join()`

When you're working with files and folders in Python, you’ll often need to build complete paths from smaller parts — like combining a base directory with a filename.

Instead of writing paths as long strings, Python provides a better tool: `os.path.join()`.

### Why Use `os.path.join()`?

Manually typing out full paths like this:

```python
file_path = "/home/user/data/simulation/input.txt"
```

might seem fine, but it’s **fragile and system-dependent**:

* You might **mistype** a slash (`/` or `\`)
* It **won’t work across platforms** (e.g., Windows vs Linux)
* It’s **hard to maintain** or reuse if directory names change

Using `os.path.join()` solves all of that:


In [36]:
base_dir = "/home/user/data/simulation"
filename = "input.txt"

file_path = os.path.join(base_dir, filename)
print(file_path) # it even fixes all the / and \

/home/user/data/simulation/input.txt


In [37]:
project_path = os.path.join('/home/jupyter', 'CommunityData', 'OpenSees')
print("Project path:", project_path)

Project path: /home/jupyter/CommunityData/OpenSees


Using `os.path.join()` helps your scripts work reliably **across platforms**, and makes your code easier to maintain, debug, and reuse — especially in Jupyter notebooks, HPC workflows, or automation pipelines.

### Troubleshooting common mistakes

* **~ is not automatically expanded** in *os.path.join*.

* **os.chdir** only affects the **current Python process**. It doesn’t change your system’s terminal session.

* Mixing hard-coded slashes (*'/'* or *'\\'*) makes scripts fragile across systems. Always use `os.path.join`.


In [38]:
os.path.join('~', 'MyData')  # WRONG: stays literally as ~/MyData

'~/MyData'

In [39]:
os.path.exists(os.path.join('~', 'MyData'))

False

In [40]:
# Instead:
os.path.join(os.path.expanduser('~'), 'MyData')  # CORRECT

'/home/jupyter/MyData'

In [41]:
os.path.exists(os.path.join(os.path.expanduser('~'), 'MyData'))

True

## Split Paths with os.path.split()

When working with file paths in Python, it's often useful to **break them apart into their components**—for example, separating a filename from its directory, or stripping off a file extension. This is especially important when building automated workflows that need to modify, move, or analyze files dynamically. The `os.path` module provides a set of reliable and portable tools for **splitting paths** into meaningful parts, so you can easily manage files without manually parsing strings. Below are some common functions that make this easy and consistent across different operating systems.

Use these when you want to programmatically handle parts of a path—e.g., checking extensions, extracting filenames, or navigating up directories.

###  `os.path.split(path)`

Splits a path into **two parts**: the **head** (directory path) and the **tail** (final component like filename or last folder).

In [42]:
path = '/home/jupyter/Work/myfile.tcl'
head, tail = os.path.split(path)
print(head)  # → /home/jupyter/Work
print(tail)  # → myfile.tcl

/home/jupyter/Work
myfile.tcl


### `os.path.splitext(path)`

Splits the path into the **filename and extension**.

In [43]:
path = '/home/jupyter/Work/myfile.tcl'
head, tail = os.path.split(path)
print('head: ',head)  # → /home/jupyter/Work
print('tail: ',tail)  # → myfile.tcl

filename, ext = os.path.splitext(tail)
print('filename: ',filename)  # → model
print('ext: ',ext)       # → .inp

head:  /home/jupyter/Work
tail:  myfile.tcl
filename:  myfile
ext:  .tcl


### `os.path.basename(path)`

Returns just the **last component** of the path (like the filename).

In [44]:
os.path.basename('/home/jupyter/Work/myfile.tcl')  

'myfile.tcl'

### `os.path.dirname(path)`

Returns just the **directory part**, without the final file or folder.

In [45]:
os.path.dirname('/home/jupyter/Work/myfile.tcl')  

'/home/jupyter/Work'

### os.path.normpath()

Removes the trailing slash (/) if there is one, so basename() returns the actual last folder name rather than an empty string.

In [46]:
path = "/home/user/projects/myproject/"
normpath = os.path.normpath(path)
print('normpath: ',normpath)  # → myproject

normpath:  /home/user/projects/myproject


### os.path.basename() (for strings)
Returns the very last folder in a path (also called the basename of the directory path).

In [47]:
path = "/home/user/projects/myproject/"
last_folder = os.path.basename(os.path.normpath(path))
print('last_folder:',last_folder)  # → myproject

last_folder: myproject


### pathlib path.name
it is more modern & flexible

In [48]:
from pathlib import Path

path = Path("/home/user/projects/myproject/")
last_folder = path.name
print(last_folder)  # → myproject


myproject


---

## Modern alternative: *pathlib*

Python’s **pathlib** module (available since 3.4) provides an **object-oriented way** to handle paths. It does everything *os.path* does, but often more elegantly.


In [49]:
from pathlib import Path

# Home directory
home = Path.home()
print(home)  # /home/jupyter

# Build a path
project = home / 'CommunityData' / 'OpenSees'
print(project)  # /home/jupyter/MyData/OpenSees

# List files
for file in project.iterdir():
    print(file)

# Absolute path
print(project.resolve())

/home/jupyter
/home/jupyter/CommunityData/OpenSees
/home/jupyter/CommunityData/OpenSees/TrainingMaterial
/home/jupyter/CommunityData/OpenSees


###  Path.resolve() Method

The `resolve()` method seen in the cell above is part of Python’s modern `pathlib` module, and it’s used to:
**Get the absolute, canonical path of a file or directory.**

When you call `.resolve()` on a `Path` object:

1. It **converts a relative path to an absolute path**
2. It **resolves any symbolic links (symlinks)**
3. It **cleans up `..` and `.` from the path**

---

###  Example

```python
from pathlib import Path

p = Path("myfolder/../data/file.txt")
print(p.resolve())
```

This might return:

```
/home/user/data/file.txt
```

Even though the original path included `../`, `resolve()` normalizes it.

###  Why Use It?

* Ensures the path is **absolute**
* Makes it easier to **compare paths** reliably
* Ensures you're working with the **actual file location**, especially if symlinks are involved

###  Note

* If the file doesn’t exist, `resolve()` may raise an error (in Python < 3.6), or simply return the cleaned-up path (in newer versions).
* You can use `resolve(strict=False)` to avoid errors if the file doesn’t exist.


###  Summary on resolve()

Use `.resolve()` when you want the **true, absolute location** of a file or folder, and want to clean up messy or relative path expressions in a reliable, cross-platform way.


## Quick cheat sheet: *os.path* vs *pathlib*
compact cheat sheet box comparing os.path to pathlib for the most common operations, so your readers can see exactly how they map line by line.

Make sure to add this at the top if you haven’t already:

```python
from pathlib import Path
import os
```

| Operation               | **os.path** & **os**     | pathlib                                                                      |
| ----------------------- | ------------------------ | ---------------------------------------------------------------------------- |
| Get current directory   | os.getcwd()             | Path.cwd()                                                                   |
| Get home directory      | os.path.expanduser('~') | Path.home()                                                                  |
| Join paths              | os.path.join(a, b, c)   | Path(a) / b / c                                                              |
| Make absolute path      | os.path.abspath(path)   | Path(path).resolve()                                                         |
| List files in directory | os.listdir(path)        | Path(path).iterdir()                                                         |
| Change directory        | os.chdir(path)          | *(no direct change, use Path objects instead to keep code clean and explicit)* |
| Check if path exists    | os.path.exists(path)                | Path(path).exists()              |
| Create a directory      | os.mkdir('new_dir')                 | Path('new_dir').mkdir()          |



### Examples

In [50]:
# os.path
import os
os.path.join('/home', 'jupyter', 'MyData')
os.path.expanduser('~')
os.path.abspath('some/file.txt')

'/home/jupyter/MyData/some/file.txt'

In [51]:
# pathlib
from pathlib import Path
Path('/home') / 'jupyter' / 'MyData'
Path.home()
Path('some/file.txt').resolve()

PosixPath('/home/jupyter/MyData/some/file.txt')

## What is **posixpath**?

**posixpath** is an **internal Python module** that implements the functions of *os.path* **specifically for POSIX-style systems**, like Linux and macOS (and broadly, anything UNIX-like).

When you do:

```python
import os
os.path.join('a', 'b')
```

you’re actually using **os.path**, which is **an alias that points to the appropriate module for your operating system**.

* On **Linux or macOS**, *os.path* is backed by *posixpath*.
* On **Windows**, *os.path* is backed by *ntpath*.

So when you import *os* or *os.path*, Python quietly does:

```python
import posixpath as os.path  # on Linux/macOS
import ntpath as os.path     # on Windows
```

This means *os.path* automatically uses the correct conventions for your system’s paths:

* / separators on Linux/macOS (via **posixpath**).
* \  separators on Windows (via **ntpath**).

---

###  Should I ever import *posixpath* directly?

 **No — almost never.**
You should always use **os.path** (or better, **pathlib**) so your code stays **portable**.

If you import **posixpath** directly, your code will break on Windows because *posixpath* hardcodes */* conventions.

---

###  Summary

| Module      | What it does                                                             | Typical use                                                  |
| ----------- | ------------------------------------------------------------------------ | ------------------------------------------------------------ |
| **os.path**   | Portable, automatically selects *posixpath* or *ntpath* based on your OS | Always recommended                                         |
| **posixpath** | Hardcoded for POSIX paths (/)                                          | Only for very specialized internal or cross-platform logic |
| **ntpath**    | Hardcoded for Windows paths (\)                                        | Same, rarely used directly                                 |



## Choosing the Right Library for Files and Paths in Python

Python offers **multiple libraries** for handling files and paths — `os`, `os.path`, `shutil`, and `pathlib` — and it’s not always obvious which one to use.

Here's a breakdown to help you understand **when and why** to use each:


| Library   | Best For                                          | Style                     | Returns        | Notes                                 |
| --------- | ------------------------------------------------- | ------------------------- | -------------- | ------------------------------------- |
| `os`      | Changing directories, running shell commands      | Procedural (older)        | Strings        | Works well with `os.path`             |
| `os.path` | Portable path operations (joining, expanding `~`) | Procedural (older)        | Strings        | Platform-independent path logic       |
| `shutil`  | Copying, moving, or deleting **files/folders**    | Procedural (higher-level) | N/A            | Useful for workflows and automation   |
| `pathlib` | Modern, object-oriented path handling             | Object-oriented           | `Path` objects | Easier, cleaner syntax for many tasks |

##  `os.path` vs `pathlib`

Both are great, but `pathlib` is **newer, more readable**, and recommended for most new code. Here’s a quick comparison:

### `os.path` Style:

In [52]:
import os
path = os.path.join(os.path.expanduser('~'), 'myfolder', 'file.txt')
print(path)

/home/jupyter/myfolder/file.txt


### `pathlib` Style:

In [53]:
from pathlib import Path
path = Path.home() / 'myfolder' / 'file.txt'
print(path)

/home/jupyter/myfolder/file.txt


`pathlib` makes paths act like objects — you can read, write, check existence, etc., directly on the path.

### So When Should You Use Each?

| Use Case                                   | Recommended Library           |
| ------------------------------------------ | ----------------------------- |
| Writing new scripts with paths             | `pathlib` ✅                   |
| Interfacing with older code or libraries   | `os.path`                     |
| Changing directories or accessing env vars | `os`                          |
| Running shell commands                     | `os.system()` or `subprocess` |
| Copying/moving/removing files and folders  | `shutil` ✅                    |
| Creating file-processing workflows         | `shutil` + `pathlib`          |